## Predicting Fraudulent Transactions
The aim of the project is to build a machine learning model to predict whether a transaction is fradulent based on transaction and identity features. 


## Why is this important?
Predicting fraudulent transactions is important to protect customers by prevent such transactions from taking place. 

In [1]:
# to handle datasets
import numpy as np
import pandas as pd

# for machine learning
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer

# for evaluation 
from sklearn.metrics import roc_auc_score

In [2]:
train_transaction_cols = ['TransactionID', 'C3', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V27', 'V28', 'V32', 'V98', 'V116', 'V117', 'V118', 'V119', 'V120', 'V153', 'V154', 'V157', 'V158', 'V235', 'V284', 'V286', 'V297', 'V300', 'V301', 'V302', 'V303', 'V304', 'V305', 'V325', 'V327', 'V328', 'TransactionDT', 'TransactionAmt', 'C1', 'C2', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'V95', 'V96', 'V97', 'V99', 'V100', 'V101', 'V102', 'V126', 'V127', 'V145', 'V166', 'V279', 'V280', 'V285', 'V287', 'V290', 'V291', 'V292', 'V293', 'V294', 'V295', 'V298', 'V299', 'V306', 'V307', 'V308', 'V309', 'V310', 'V311', 'V312', 'V313', 'V316', 'V317', 'V318', 'V319', 'V320', 'V321', 'V322', 'V323', 'V324', 'V326', 'V329', 'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338', 'V339', 'R_emaildomain', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'ProductCD', 'isFraud']

train_identity_cols = ['TransactionID', 'id_08', 'id_13', 'id_17', 'id_19', 'id_20', 'id_21', 'id_26', 'id_16', 'id_27', 'DeviceInfo']

In [3]:
# load data
train_transaction = pd.read_csv("../data/train_transaction.csv", usecols=train_transaction_cols)
train_identity = pd.read_csv("../data/train_identity.csv", usecols=train_identity_cols)

# merge the dataframe
dataframe = pd.merge(train_transaction,
                     train_identity,
                     how="left",
                     on="TransactionID")

In [4]:
dataframe.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card5,addr1,...,id_08,id_13,id_16,id_17,id_19,id_20,id_21,id_26,id_27,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,142.0,315.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,102.0,325.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,166.0,330.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,117.0,476.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,102.0,420.0,...,NaN,NaN,NotFound,166.0,542.0,144.0,NaN,NaN,NaN,SAMSUNG SM-G892A Build/NRD90M


In [5]:
# convert numeric features to object
# to follow the documentation 
numeric_cat_cols = ["card1", "card2", "card3", "card5", "addr1", "addr2", "id_13",
                    "id_17", "id_19", "id_20", "id_21", "id_26"]

cat_cols = list(dataframe.select_dtypes(include=['object']).columns)
cat_cols += numeric_cat_cols 

dataframe[cat_cols] = dataframe[cat_cols].astype("O")

# collecting all of the numerical features 
num_cols = [x for x in dataframe.select_dtypes(include=['number']).columns
            if x not in cat_cols
            if x not in "isFraud"]

# collecting all of the features with missing values
features_with_na = [x for x in dataframe.columns if dataframe[x].isnull().sum() > 0]

# determine percentage of missing values (expressed as decimals)
# and display the result ordered by % of missin data
missing_vals_df = pd.DataFrame(
    dataframe[features_with_na].isnull().mean().sort_values(ascending=False),
    columns=["percentage"]
)

# seperating features with missing
# values to categorical and numeric
cat_feat_with_na = [x for x in cat_cols if x in features_with_na]
num_feat_with_na = [x for x in num_cols if x in features_with_na]

# features to remove
drop_features = [col for col in 
                 list(missing_vals_df.iloc[np.where(missing_vals_df.loc[cat_feat_with_na, 'percentage'] > 0.5)].index)]

drop_features += [col for col in
                  list(missing_vals_df.iloc[np.where(missing_vals_df.loc[num_feat_with_na, 'percentage'] > 0.5)].index)]

# defining the different features 
discrete_features = [x for x in num_cols 
                     if len(dataframe[x].unique()) < 20
                     and x not in drop_features]

continuous_features = [x for x in num_cols
                       if x not in discrete_features + drop_features + ["TransactionID"]]

high_cardinality_cats = ["R_emaildomain", "card1", "card2", "card3",
                         "card5", "addr1", "addr2", "id_13", "id_17", "id_19", "id_20",
                         "id_21", "id_26"]

categorical_features = [x for x in cat_cols if x not in drop_features + high_cardinality_cats]

all_features = discrete_features + continuous_features + high_cardinality_cats + categorical_features

impute_freq = high_cardinality_cats + categorical_features + discrete_features

cat_codes_cols = high_cardinality_cats + categorical_features

In [24]:
from collections import Counter
from typing import List, Union

import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

class AggregateCategorical(BaseEstimator, TransformerMixin):
    """
    Reduces the cardinality of categorical features
    Credit: Raj Sangani- https://bit.ly/3BSxdTX

    Parameters
    ----------
    features: str or list
        The feature(s) with high cardinality that we want to
        aggregate

    threshold: float

    Methods
    ----------
    fit:
        The transformer will not learn from any parameter

    transform:
        Drops the explicitly selected features
    """

    def __init__(self, features: List[Union[str, int]], threshold: float = 0.75):
        if not isinstance(features, list) or len(features) == 0:
            raise ValueError("Was expecting a list of features")
        self.features = features
        self.threshold = threshold

    def fit(self, X: pd.DataFrame, y: pd.Series = None):
        return self

    def transform(self, X: pd.DataFrame):
        # find the threshold value using the percentage and number of instances
        threshold_value = int(self.threshold * len(X))
        df = X.copy()

        for col in self.features:
            counts = Counter(df[col])
            s = 0
            category_list = []
            # loop through the category name and its corresponding frequency
            for i, j in counts.most_common():
                # add the frequency to the sum
                s += counts[i]
                category_list.append(i)

                if s >= threshold_value:
                    break
            # replace all instances not in our new categories by other
            category_list.append("other")
            df[col] = df[col].apply(lambda x: x if x in category_list else "other")
        return df


class MostFrequentImputer(BaseEstimator, TransformerMixin):
    """
    A wrapper around `SimpleImputer` to return data frames with columns.
    Credit: https://bit.ly/3r2N40k
    """
    def __init__(self, features):
        if not isinstance(features, list) or len(features) == 0:
            raise ValueError("Was expecting a list of features")
            
        self.features = features
        
    def fit(self, X, y=None):
        self.imputer_dict_ = {}
        for feature in self.features:
            self.imputer_dict_[feature] = X[feature].mode()[0]
        return self

    def transform(self, X):
        X = X.copy()
        for feature in self.features:
            X[feature] = X[feature].fillna(self.imputer_dict_[feature])
        return X
    
class MeanImputer(BaseEstimator, TransformerMixin):
    """
    Numerical missing value imputer.
    Credit: https://bit.ly/3r2N40k
    """

    def __init__(self, features):
        if not isinstance(features, list) or len(features) == 0:
            raise ValueError("Was expecting a list of features")
        
        self.features = features

    def fit(self, X, y=None):
        # persist mode in a dictionary
        self.imputer_dict_ = {}
        for feature in self.features:
            self.imputer_dict_[feature] = X[feature].mean()
        return self

    def transform(self, X):
        X = X.copy()
        for feature in self.features:
            X[feature].fillna(self.imputer_dict_[feature], inplace=True)
        return X
    
class CategoryConverter(BaseEstimator, TransformerMixin):
    def __init__(self, features):
        if not isinstance(features, list) or len(features) == 0:
            raise ValueError("Was expecting a list of features")
            
        self.features = features
        
    def fit(self, X, y=None):
        self.codes_ = {}
        for feature in self.features:
            X[feature] = X[feature].astype("category")
            self.codes_[feature] = dict(zip(X[feature].values, X[feature].cat.codes))
        return self
    
    def transform(self, X):
        X = X.copy()
        for feature in self.features: 
            X[feature] = X[feature].map(self.codes_[feature])
        return X
    

### Building a classifier without an ML pipeline

In [35]:
df1_ = dataframe.copy()

# impute mode to high cardinality, categorical features,
# and discrete features
imp_most_frequent = MostFrequentImputer(features=impute_freq)
df1_ = imp_most_frequent.fit_transform(df1_)

# aggregate the high cardinality categoricals so they have less dimensions
aggregate_categoricals = AggregateCategorical(high_cardinality_cats)
df1_ = aggregate_categoricals.fit_transform(df1_)

# convert all categorical features to type category and use codes
transform_dtype = CategoryConverter(features=cat_codes_cols)
df1_ = transform_dtype.fit_transform(df1_)

# impute the mean to the continuous features
imp_mean = MeanImputer(features=continuous_features)
df1_ = imp_mean.fit_transform(df1_)

X = df1_[all_features]
y = df1_["isFraud"]

# split into train and test
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.33,
    stratify=y,
    random_state=25
)

# modelling
random_forest = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=25)
random_forest.fit(X_train, y_train)

train_preds = random_forest.predict_proba(X_train)
y_preds = random_forest.predict_proba(X_test)

preds_roc_auc_ = roc_auc_score(y_test, y_preds[:, 1])
train_roc_auc_ = roc_auc_score(y_train, train_preds[:, 1])

print(f"train ROC_AUC:{train_roc_auc_}\n\
test ROC_AUC: {preds_roc_auc_}")

train ROC_AUC:1.0
test ROC_AUC: 0.912609226302252


### Building a classifier with ML Pipeline
Using `Pipeline`

In [45]:
# pipeline transformations
process_pipe = Pipeline(
    [
        (
            "most_frequent_imputer", 
            MostFrequentImputer(features=impute_freq)
        ), 
        (
            "aggregate_high_cardinality_features",
            AggregateCategorical(features=high_cardinality_cats),
        ), 
        (
            "get_categorical_codes",
            CategoryConverter(features=cat_codes_cols),
        ),
        (
            "mean_imputer",
            MeanImputer(features=continuous_features),
        )
    ]
)

In [46]:
df2_ = dataframe.copy()

# seperate training features and target features
# perform all processing to training features
X = process_pipe.fit_transform(df2_[all_features])
y = df2_["isFraud"]

# split into train and test data
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.33,
    stratify=y,
    random_state=25
)

# modelling
random_forest = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=25)
random_forest.fit(X_train, y_train)

train_preds = random_forest.predict_proba(X_train)
y_preds = random_forest.predict_proba(X_test)

preds_roc_auc_ = roc_auc_score(y_test, y_preds[:, 1])
train_roc_auc_ = roc_auc_score(y_train, train_preds[:, 1])

print(f"train ROC_AUC:{train_roc_auc_}\n\
test ROC_AUC: {preds_roc_auc_}")

train ROC_AUC:1.0
test ROC_AUC: 0.912609226302252
